<a href="https://colab.research.google.com/github/Praneeth-18/RAG/blob/main/Part_2_HW_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***This Notebook runs only on L4***

In [ ]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

In [ ]:
!pip install kagglehub

# Then you import kagglehub and use its login().
import kagglehub
kagglehub.login()

In [ ]:
!pip install --upgrade keras-nlp
!pip install --upgrade keras

In [ ]:
import keras_nlp

gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")

Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'tokenizer.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...


In [ ]:
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import re

def format_poem(text):
    # Extract text starting from "Answer:" onwards
    text = text.split("Answer:", 1)[-1]

    # Remove any "Step x/y" patterns
    text = re.sub(r'Step \d+/\d+', '', text)

    # Replace any sequence of spaces with a single space
    text = re.sub(r'\s+', ' ', text).strip()

    # Split the string at the specified punctuation marks followed by a space
    # This keeps the punctuation marks at the end of each line
    lines = re.split(r'(?<=[,.:;]) ', text)

    # Join the lines with a newline character to format as a poem
    poem = '\n'.join(lines)

    return poem

def tokenize(s):
    return re.split('\s+', s)

def untokenize(ts):
    return ' '.join(ts)

def equalize(s1, s2):
    # This version of equalize doesn't highlight differences, just aligns for side-by-side comparison
    l1 = tokenize(s1)
    l2 = tokenize(s2)
    return untokenize(l1), untokenize(l2)

def insert_newlines(string, every=64, window=10):
    result = []
    from_string = string
    while len(from_string) > every:
        cut_off = every
        while cut_off > every - window and from_string[cut_off - 1] != ' ':
            cut_off -= 1
        result.append(from_string[:cut_off])
        from_string = from_string[cut_off:].lstrip()
    result.append(from_string)
    return result

def show_comparison(s1, s2, title1='Text 1', title2='Text 2', width=40, margin=10, sidebyside=True):
    s1, s2 = equalize(s1, s2)

    if sidebyside:
        s1_lines = insert_newlines(s1, width, margin)
        s2_lines = insert_newlines(s2, width, margin)
        max_lines = max(len(s1_lines), len(s2_lines))

        # Pad shorter list with empty strings to match lengths
        s1_lines.extend([''] * (max_lines - len(s1_lines)))
        s2_lines.extend([''] * (max_lines - len(s2_lines)))

        # Print titles
        print(title1.center(width) + ' | ' + title2.center(width) + ' |')

        for i in range(max_lines):
            lft = s1_lines[i].ljust(width)
            rgt = s2_lines[i].ljust(width)
            print(lft + ' | ' + rgt + ' | ')
    else:
        print(title1 + '\n' + s1)
        print(title2 + '\n' + s2)


In [ ]:
prompt = "Write a Poem titled If...."
result1 = format_poem(gemma_lm.generate(prompt, max_length=128))

In [ ]:
print(format_poem(result1))

Write a Poem titled If....
If you could write a poem about your life,
what would you write about? Write a poem about your life.
Write a poem about your life.
Write a poem about your life.
Write a poem about your life.
Write a poem about your life.
Write a poem about your life.
Write a poem about your life.
Write a poem about your life.
Write a poem about your life.
Write a poem about your life.
Write a poem about your life.
Write a poem about your life.
Write a poem about your life.


# Fine Tuning on Rudyard Kiplings' Poems

In [ ]:
import pandas as pd
Rudyard_poems = pd.read_csv('/content/drive/MyDrive/257rp/RP_df.csv')

In [ ]:
Rudyard_poems

,Title,Poem,Poem_for_Human
0,If....,if you can keep your head when all about you a...,\n\tIf you can keep your head when all about y...
1,The Law Of The Jungle,now this is the law of the jungle as old and a...,Now this is the Law of the Jungle — as old and...
2,The Way Through The Woods,they shut the road through the woods seventy y...,\r\n\tThey shut the road through the woods\r\n...
3,Boots,infantry columns we are foot—slog—slog—slog—sl...,INFANTRY COLUMNS\n\nWe're foot—slog—slog—slog—...
4,The Camels Hump,the camel hump is an ugly lump which well you ...,The Camel's hump is an ugly lump \nWhich well ...
5,Fuzzy-Wuzzy,we have fought with many men acrost the seas a...,"We've fought with many men acrost the seas,\n ..."
6,The Last of the Light Brigade,there were thirty million english who talked o...,There were thirty million English who talked o...
7,Dirge Of The Dead Sisters,who recalls the twilight and the ranged tents ...,Who recalls the twilight and the ranged tents ...
8,Sussex,god gave all men all earth to love but since o...,"GOD gave all men all earth to love,\n But s..."
9,The Hyaenas,after the burial parties leave and the baffled...,After the burial-parties leave\n And the baff...


In [ ]:
data_RK =[]
for i in range(len(Rudyard_poems)):
  title = Rudyard_poems['Poem'][i]
  poem = Rudyard_poems['Poem_for_Human'][i]
  template = f"Instruction:\n Write a poem titled {title} in Rudyard Kiplings' style and your response should be in proper poem format with proper beautiful spacing tabs and line breaks \n\nResponse:\n{poem}"
  data_RK.append(template)

In [ ]:
data_RK

['Instruction:\n Write a poem titled if you can keep your head when all about you are losing theirs and blaming it on you if you can trust yourself when all men doubt you but make allowance for their doubting too if you can wait and not be tired by waiting or being lied about do not deal in lies or being hated do not give way to hating and yet do not look too good nor talk too wise if you can dream and not make dreams your master if you can think and not make thoughts your aim if you can meet with triumph and disaster and treat those two impostors just the same if you can bear to hear the truth you have spoken twisted by knaves to make trap for fools or watch the things you gave your life to broken and stoop and build them up with worn out tools if you can make one heap of all your winnings and risk it on one turn of pitch and toss and lose and start again at your beginnings and never breathe word about your loss if you can force your heart and nerve and sinew to serve your turn long a

In [ ]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,507,536,384 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,536,384 (9.34 GB)

 Trainable params: 1,363,968 (5.20 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

In [ ]:
import tensorflow
import keras
import torch
import keras_nlp

gemma_lm.preprocessor.sequence_length = 512
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data_RK, epochs=10, batch_size=1)

Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 50s 253ms/step - loss: 2.7013 - sparse_categorical_accuracy: 0.5376
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 253ms/step - loss: 2.6953 - sparse_categorical_accuracy: 0.5388
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 254ms/step - loss: 2.6898 - sparse_categorical_accuracy: 0.5393
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 254ms/step - loss: 2.6828 - sparse_categorical_accuracy: 0.5396
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 254ms/step - loss: 2.6743 - sparse_categorical_accuracy: 0.5415
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 255ms/step - loss: 2.6639 - sparse_categorical_accuracy: 0.5426
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 255ms/step - loss: 2.6519 - sparse_categorical_accuracy: 0.5446
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 256ms/step - loss: 2.6383 - sparse_categorical_accuracy: 0.5470
Epoch 9/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 256ms/step - loss: 2.6236 - sparse_categorical_accuracy: 0.5498
Epoch 10/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 256ms/step -

In [ ]:
prompt = "Write a Poem titled If.... in Rudyard Kiplings' Style and your response should be in proper poem format with proper beautiful spacing tabs and line breaks"
result2 = gemma_lm.generate(prompt, max_length=128)

In [ ]:
print(format_poem(result2))

If I could fly like a bird,
I would soar above the clouds and see the world from a different perspective.
I would see the beauty of the mountains,
the majesty of the oceans,
and the wonder of the stars.
I would feel the freedom of flight,
the exhilaration of soaring,
and the joy of being free.
If I could walk like a tree,
I would grow strong and tall,
reaching for the sky and


In [ ]:
show_comparison(format_poem(result1), format_poem(result2), title1 = 'Initial', title2 = 'Fine Tuned on Rudyard Kiplings Poems')

                Initial                  |   Fine Tuned on Rudyard Kiplings Poems   |
Write a Poem titled If.... If you could  | If I could fly like a bird, I would      | 
write a poem about your life, what       | soar above the clouds and see the world  | 
would you write about? Write a poem      | from a different perspective. I would    | 
about your life. Write a poem about      | see the beauty of the mountains, the     | 
your life. Write a poem about your       | majesty of the oceans, and the wonder    | 
life. Write a poem about your life.      | of the stars. I would feel the freedom   | 
Write a poem about your life. Write a    | of flight, the exhilaration of soaring,  | 
poem about your life. Write a poem       | and the joy of being free. If I could    | 
about your life. Write a poem about      | walk like a tree, I would grow strong    | 
your life. Write a poem about your       | and tall, reaching for the sky and       | 
life. Write a poem about your life.      |  

The fine tuned model performed Significantly better.

In [ ]:
prompt = "Write a Poem on topic Football in Rudyard Kiplings' Style and your response should be in proper poem format with proper beautiful spacing tabs and line breaks"
result3 = format_poem(gemma_lm.generate(prompt, max_length=128))

In [ ]:
print(format_poem(result3))

Football is a game that brings people together,
and it's a game that can bring joy and excitement.
It's a game that can bring people together,
and it's a game that can bring joy and excitement.
It's a game that can bring people together,
and it's a game that can bring joy and excitement.
It's a game that can bring people


# Without fine tuning on T S Eliots' Style

In [ ]:
prompt = "Write a Poem titled If.... in T S Eliots' Style and your response should be in proper poem format with proper beautiful spacing tabs and line breaks"
result4 = gemma_lm.generate(prompt, max_length=128)

In [ ]:
print(format_poem(result4))

If I could be a butterfly,
I would fly through the clouds and the blue skies.
I would flutter my wings and make a sound,
and I would be free from all the cares and the woes.
If I could be a butterfly,
I would fly away from the world,
and I would be free from all the troubles and the strife.
I would be a butterfly,
and I would be


# Fine Tuning on T S Eliots' Poems

In [ ]:
data2 = pd.read_csv('/content/drive/MyDrive/257rp/T_S_Eliot.csv')

In [ ]:
data2.drop('Unnamed: 0',axis=1,inplace=True)
data2

,Title,Poem,Poem_for_Human
0,The Naming Of Cats,the naming of cats is difficult matter it is n...,"The Naming of Cats is a difficult matter,\nIt ..."
1,The Hollow Men,mistah kurtz he dead penny for the old guy we ...,Mistah Kurtz-he dead\n A penny for...
2,Macavity: The Mystery Cat,macavity mystery cat he is called the hidden p...,Macavity's a Mystery Cat: he's called the Hidd...
3,The Waste Land,part burial of the dead april is the cruelest ...,Part 1 - *Burial of the Dead*\nApril is the cr...
4,The Old Gumbie Cat,have gumbie cat in mind her name is jennyanydo...,"I have a Gumbie Cat in mind, her name is Jenny..."
5,The Love Song Of J. Alfred Prufrock,s’io credesse che mia risposta fosse persona c...,S’io credesse che mia risposta fosse \n A per...
6,Eyes That Last I Saw In Tears,eyes that last saw in tears through division h...,Eyes that last I saw in tears\r\nThrough divis...
7,The Journey Of The Magi,cold coming we had of it just the worst time o...,"A cold coming we had of it,\nJust the worst t..."
8,Portrait Of A Lady,thou hast committed— fornication but that was ...,Thou hast committed—\n Fornication: but that ...
9,Morning At The Window,they are rattling breakfast plates in basement...,They are rattling breakfast plates in basement...


In [ ]:
data_TSE =[]
for i in range(len(data2)):
  title = data2['Poem'][i]
  poem = data2['Poem_for_Human'][i]
  template = f"Instruction:\n Write a poem titled {title} in TS Eliots's style and your response should be in proper poem format with proper beautiful spacing tabs and line breaks \n\nResponse:\n{poem}"
  data_TSE.append(template)

In [ ]:
data_TSE

["Instruction:\n Write a poem titled the naming of cats is difficult matter it is not just one of your holiday games you may think at first am as mad as hatter when tell you cat must have three different names first of all there is the name that the family use daily such as peter augustus alonzo or james such as victor or jonathan george or bill bailey— all of them sensible everyday names there are fancier names if you think they sound sweeter some for the gentlemen some for the dames such as plato admetus electra demeter— but all of them sensible everyday names but tell you cat needs name that is particular name that is peculiar and more dignified else how can he keep up his tail perpendicular or spread out his whiskers or cherish his pride of names of this kind can give you quorum such as munkustrap quaxo or coricopat such as bombalurina or else jellylorum names that never belong to more than one cat but above and beyond there is still one name left over and that is the name that you

In [ ]:
# Limit the input sequence length to 512 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 512
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data_TSE, epochs=10, batch_size=1)

Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 44s 251ms/step - loss: 3.1847 - sparse_categorical_accuracy: 0.4429
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 251ms/step - loss: 3.1649 - sparse_categorical_accuracy: 0.4432
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 251ms/step - loss: 3.1509 - sparse_categorical_accuracy: 0.4432
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 253ms/step - loss: 3.1376 - sparse_categorical_accuracy: 0.4467
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 252ms/step - loss: 3.1245 - sparse_categorical_accuracy: 0.4498
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 253ms/step - loss: 3.1110 - sparse_categorical_accuracy: 0.4513
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 253ms/step - loss: 3.0973 - sparse_categorical_accuracy: 0.4541
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 253ms/step - loss: 3.0828 - sparse_categorical_accuracy: 0.4556
Epoch 9/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 255ms/step - loss: 3.0678 - sparse_categorical_accuracy: 0.4574
Epoch 10/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 254ms/step -

In [ ]:
prompt = "Write a Poem titled if .... in TS ELiots' Style and your response should be in proper poem format with proper beautiful spacing tabs and line breaks"
result5 = gemma_lm.generate(prompt, max_length=128)

In [ ]:
print(format_poem(result5))

If I could be a tree,
I would stand tall and strong,
With roots deep in the earth,
and branches reaching for the sky.
I would be a home for birds and animals,
A place of refuge and shelter from the storm.
I would be a place of beauty and peace,
A place where all can come and find their peace.
If I could be a tree,
I would be a symbol of strength and resilience,
A


In [ ]:
show_comparison(format_poem(result4), format_poem(result5),title1 = 'Initial', title2 = 'Fine Tuned on T S Eliots Poems')

                Initial                  |      Fine Tuned on T S Eliots Poems      |
If I could be a butterfly, I would fly   | If I could be a tree, I would stand      | 
through the clouds and the blue skies.   | tall and strong, With roots deep in the  | 
I would flutter my wings and make a      | earth, and branches reaching for the     | 
sound, and I would be free from all the  | sky. I would be a home for birds and     | 
cares and the woes. If I could be a      | animals, A place of refuge and shelter   | 
butterfly, I would fly away from the     | from the storm. I would be a place of    | 
world, and I would be free from all the  | beauty and peace, A place where all can  | 
troubles and the strife. I would be a    | come and find their peace. If I could    | 
butterfly, and I would be                | be a tree, I would be a symbol of        | 
                                         | strength and resilience, A               | 


In [ ]:
prompt = "Write a Poem on topic Football in T S Eliots' Style and your response should be in proper poem format with proper beautiful spacing tabs and line breaks"
result6 = format_poem(gemma_lm.generate(prompt, max_length=128))

In [ ]:
print(format_poem(result6))

Football is a game that brings people together,
It's a game that unites us all,
no matter our race or creed.
It's a game that brings joy and laughter,
It's a game that brings us together,
no matter the score.
Football is a game that brings us together,
It's a game that brings us together,
no matter the score.


In [ ]:
show_comparison(format_poem(result3), format_poem(result6),title1 = 'Football in Rudyard Kiplings Style', title2 = 'Football in T S Eliots Style')

   Football in Rudyard Kiplings Style    |       Football in T S Eliots Style       |
Football is a game that brings people    | Football is a game that brings people    | 
together, and it's a game that can       | together, It's a game that unites us     | 
bring joy and excitement. It's a game    | all, no matter our race or creed. It's   | 
that can bring people together, and      | a game that brings joy and laughter,     | 
it's a game that can bring joy and       | It's a game that brings us together, no  | 
excitement. It's a game that can bring   | matter the score. Football is a game     | 
people together, and it's a game that    | that brings us together, It's a game     | 
can bring joy and excitement. It's a     | that brings us together, no matter the   | 
game that can bring people               | score.                                   | 
